In [1]:
import yfinance as yf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
tickers=['AAPL','GOOG','C','DIS','F','MSFT','MS','GME','TSLA','AMZN']
start=dt.datetime(2019,1,1)
end=dt.datetime.now()

In [3]:
def getData(stocks, start, end):
    stockData = yf.download(stocks,start,end)['Adj Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

In [4]:
returns,avg_returns,cov_matrix=getData(tickers, start, end)
returns.head()

[*********************100%***********************]  10 of 10 completed


,AAPL,AMZN,C,DIS,F,GME,GOOG,MS,MSFT,TSLA
Date,,,,,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,-0.099608,-0.025241,-0.018121,-0.024227,-0.01519,-0.007651,-0.028484,-0.017822,-0.036788,-0.031472
2019-01-04,0.042689,0.050064,0.048896,0.030847,0.03856,0.175019,0.053786,0.040827,0.046509,0.057697
2019-01-07,-0.002226,0.034353,0.008707,0.008667,0.02599,0.015748,-0.002167,0.009928,0.001275,0.054361
2019-01-08,0.019063,0.016612,-0.002697,0.007779,0.00965,0.021318,0.007385,-0.006234,0.007251,0.001164


In [5]:
cov_matrix

,AAPL,AMZN,C,DIS,F,GME,GOOG,MS,MSFT,TSLA
AAPL,0.000474,0.000319,0.000274,0.000230,0.000248,0.000359,0.000309,0.000284,0.000342,0.000455
AMZN,0.000319,0.000507,0.000188,0.000215,0.000186,0.000299,0.000305,0.000201,0.000316,0.000433
C,0.000274,0.000188,0.000693,0.000362,0.000452,0.000468,0.000261,0.000520,0.000258,0.000345
DIS,0.000230,0.000215,0.000362,0.000488,0.000328,0.000270,0.000235,0.000326,0.000232,0.000299
F,0.000248,0.000186,0.000452,0.000328,0.000801,0.000551,0.000231,0.000411,0.000219,0.000402
GME,0.000359,0.000299,0.000468,0.000270,0.000551,0.010259,0.000242,0.000309,0.000266,0.000713
GOOG,0.000309,0.000305,0.000261,0.000235,0.000231,0.000242,0.000410,0.000263,0.000324,0.000364
MS,0.000284,0.000201,0.000520,0.000326,0.000411,0.000309,0.000263,0.000557,0.000268,0.000322
MSFT,0.000342,0.000316,0.000258,0.000232,0.000219,0.000266,0.000324,0.000268,0.000400,0.000404
TSLA,0.000455,0.000433,0.000345,0.000299,0.000402,0.000713,0.000364,0.000322,0.000404,0.001766


In [6]:
#Expected return(mean)
avg_returns

AAPL    0.001564
AMZN    0.000401
C       0.000307
DIS     0.000099
F       0.001031
GME     0.006330
GOOG    0.000786
MS      0.001195
MSFT    0.001136
TSLA    0.003058
dtype: float64

In [7]:
#Count how many returns of each stock
count=returns.count()
count

AAPL    993
AMZN    993
C       993
DIS     993
F       993
GME     993
GOOG    993
MS      993
MSFT    993
TSLA    993
dtype: int64

In [8]:
weights = np.random.random(len(returns.columns))
weights

array([0.06469181, 0.43597269, 0.40618155, 0.64006977, 0.70433079,
       0.77017729, 0.27891178, 0.41234083, 0.41902321, 0.26483871])

In [9]:
weights = weights/np.sum(weights)
weights

array([0.01471426, 0.09916272, 0.09238667, 0.14558494, 0.16020121,
       0.17517811, 0.06343895, 0.09378761, 0.09530753, 0.06023801])

In [10]:
# Portfolio Performance
def portfolioPerformance(weights, mean_Returns, cov_Matrix):
    
    port_mean = (mean_Returns@weights)
    
    port_std = np.sqrt((weights.T@cov_Matrix@weights)) 
    
    return port_mean, port_std

In [11]:
Time=100

port_mean,port_std=portfolioPerformance(weights, avg_returns, cov_matrix)
port_std

0.02577476963376967

In [12]:
port_mean

0.0018341441150778947

In [13]:
port_std

0.02577476963376967

In [14]:
#If you want 90% VaR, you should enter alpha=10
#If you want 99% VaR, you should enter alpha=1
def var_parametric(port_mean, port_std, Time, distribution='normal', alpha=5):
    
    # because the distribution is symmetric
    if distribution == 'normal':
        VaR = (norm.ppf(alpha/100,port_mean,port_std))*np.sqrt(Time)
    
    else:
        raise TypeError("Expected distribution type 'normal'")
    
    return VaR

In [15]:
VaR =var_parametric(port_mean, port_std,Time)
VaR

-0.4056157920086684

In [16]:
norm.ppf(0.05,port_mean,port_std)*np.sqrt(Time)

-0.4056157920086684

In [17]:
InitialInvestment = 10000
print("Normal VaR 95th CI       :      ", round(InitialInvestment*-VaR,2))

Normal VaR 95th CI       :       4056.16


In [18]:
norm.cdf(1.64)

0.9494974165258963

In [19]:
norm.ppf(0.95)

1.6448536269514722

In [20]:
#Confidence interval
def CI(port_mean, port_std,n,alpha=5):
    lower=port_mean-norm.ppf(1-alpha/100)*port_std/np.sqrt(n)
    higher=port_mean+norm.ppf(1-alpha/100)*port_std/np.sqrt(n)
    return lower, higher

In [21]:
lower,higher=CI(port_mean, port_std,count[0],alpha=5)
lower

0.0004887565016763898

In [22]:
higher

0.0031795317284793994

In [23]:
print("We are 95% confidence that the daily return of our portfolio will be between", round(lower*100,2), '% and', round(higher*100,2),'%')

We are 95% confidence that the daily return of our portfolio will be between 0.05 % and 0.32 %
